In [19]:
from datetime import datetime
#vytvoření časové značky
timestamp = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
#vytvoření univerzální značky
import uuid 
id = uuid.uuid1()
import psycopg2
from rich import print

# Rozsahy kategorií
rozsahy_kategorii = {
    "N": (1.00, 1.17, 1),
    "P": (1.17, 1.22, 2),
    "F1": (1.22, 1.26, 3),
    "F2": (1.26, 1.32, 4),
    "F3": (1.32, 1.37, 5),
    "C1": (1.37, 1.41, 6),
    "C2": (1.41, 1.53, 7),
    "C3": (1.53, 2.00, 8),
    "N/A": (None, None, 99)  # Pro 0 vody
}

def urci_kategorii(vazeny_prumer):
    if vazeny_prumer is None:
        return f"Kategorie 99 (N/A)"  # Pro 0 vody

    for kod, (minimum, maximum, cislo) in rozsahy_kategorii.items():
        if minimum is not None and maximum is not None:
            if minimum <= vazeny_prumer <= maximum:
                return f"Kategorie {cislo} ({kod})"
        elif kod == "N/A" and vazeny_prumer is None:
            return f"Kategorie 99 (N/A)"

    return "Neznámá kategorie"  # Pro případ, že hodnota nespadá do žádného rozsahu

# Funkce pro určení barvy a poznámky na základě klasického měřítka
def urci_barvu_klasicke_meritko(meritko):
    meritko_cislo = int(meritko.replace("1:", "").replace(" ", ""))
    if meritko_cislo <= 10000:
        return "#006400", ""  # tm. zelená, bez poznámky
    elif meritko_cislo <= 50000:
        return "#FFA500", "Měřítko má méně přesné rozlišení."  # oranžová, poznámka
    else:
        return "#FF0000", "Měřítko velmi hrubé - Data je potřeba interpretovat obezřetně."  # červená, poznámka

# Funkce pro určení barvy a poznámky na základě měřítka v metrech na pixel
def urci_barvu_meritko_na_pixel(meritko):
    meritko_cislo = int(meritko.replace("m/px", "").replace(" ", ""))
    if meritko_cislo <= 20:
        return "#006400", ""  # tm. zelená, bez poznámky
    elif meritko_cislo <= 100:
        return "#FFA500", "Měřítko má méně přesné rozlišení."  # oranžová, poznámka
    else:
        return "#FF0000", "Měřítko velmi hrubé - Data je potřeba interpretovat obezřetně."  # červená, poznámka
        
# Funkce pro určení barvy na základě stáří roku
def urci_barvu_rok(rok):
    aktualni_rok = datetime.now().year
    rozdil = aktualni_rok - rok
    if rozdil < 3:
        return "#1a9641"  # zelená
    elif rozdil < 5:
        return "#fdae61"  # oranžová
    else:
        return "#d7191c"  # červená
        
# Přihlašovací údaje k databázi
dbname = 'DP_pokus'
user = 'postgres'
password = 'kapli4ky'
host = 'localhost'
port = '5432'  # Výchozí port PostgreSQL

try:
    # Vytvoření připojení
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    # Vytvoření kurzoru pro provádění SQL dotazů
    cursor = connection.cursor()
    print("Připojení k databázi bylo úspěšné")

     # SQL dotaz na vybraní měřítek a roků z tabulky
    sql_query = """
    SELECT zkratka_txt, datovy_zdroj, indikator_cz, meritko, last_update, source_dat FROM metadata_dat
    """
    sql_query_degradace = """
    WITH plochy_veget AS ( 
        SELECT 
            geom,
            ST_Area(geom) AS plocha 
        FROM stav_vegetace
    ),
    plochy_pudy AS (
        SELECT
            geom,
            ST_Area(geom) AS plocha
        FROM stav_pudy
    ),
    plochy_klima AS (
        SELECT
            geom,
            ST_Area(geom) AS plocha
        FROM stav_klimatu
    ),
    plochy_mgm AS (
        SELECT
            geom,
            ST_Area(geom) AS plocha
        FROM stav_intenzity_lidske_cinnosti
    )
    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema, 
        SUM(w_veget * plocha) / SUM(plocha) AS hodnota 
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL 
    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Odolnost vegetace vůči suchu' AS podrazene_tema,
        SUM(dry_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL

    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Schopnost vegetace bránit erozi' AS podrazene_tema,
        SUM(ero_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL

    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Pokryvnost vegetace' AS podrazene_tema,
        SUM(pla_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema,  
        SUM(w_soil * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Sklonitost svahu' AS podrazene_tema,
        SUM(w_slo * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Textura' AS podrazene_tema,
        SUM(w_tex * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Skeletovitost' AS podrazene_tema,
        SUM(w_fra * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Hloubka půdy' AS podrazene_tema,
        SUM(w_dep * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Propustnost' AS podrazene_tema,
        SUM(w_dra * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Matečná hornina' AS podrazene_tema,
        SUM(par_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema,  
        SUM(w_climate * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Průměrný roční úhrn srážek' AS podrazene_tema,
        SUM(w_rai * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Index sucha' AS podrazene_tema,
        SUM(w_arid * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Orientace svahu' AS podrazene_tema,
        SUM(w_asp * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema,  
        SUM(w_mgm * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Intenzita využití půdy' AS podrazene_tema,
        SUM(int * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Hustota populace' AS podrazene_tema,
        SUM(den_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Populační růst' AS podrazene_tema,
        SUM(grw_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom
    """
    cursor.execute(sql_query)
    data = cursor.fetchall()
    cursor.execute(sql_query_degradace)
    data_degradace = cursor.fetchall()


    # Zpracování dat
    for row in data:
        zkratka_txt = row[0]
        datovy_zdroj = row[1]
        indikator_cz = row[2]
        meritko = row[3]
        last_update = row[4]
        source_dat = row[5]

    
    # Seřazení a příprava dat pro tabulku náchylnosti k degradaci
    serazena_data_degradace = {}

    for row in data_degradace:
        nadrazene_tema = row[0]
        podrazene_tema = row[1]
        hodnota = row[2]
        kategorie = urci_kategorii(hodnota)

        if nadrazene_tema not in serazena_data_degradace:
            serazena_data_degradace[nadrazene_tema] = {
                'hodnota': None,
                'kategorie': None,
                'podrazene': []
            }

        if podrazene_tema == 'Celková hodnota':
            serazena_data_degradace[nadrazene_tema]['hodnota'] = hodnota
            serazena_data_degradace[nadrazene_tema]['kategorie'] = kategorie
        else:
            serazena_data_degradace[nadrazene_tema]['podrazene'].append(
                (podrazene_tema, hodnota, kategorie))

    serazene_nadrazene_degradace = sorted(
        serazena_data_degradace.items(), key=lambda x: -x[1]['hodnota'])

    # Generování HTML obsahu
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Zranitelnost krajiny</title>
        <meta charset='UTF-8'>
        <meta name='description' content='HTML stránka georeportu zranitelnosti krajiny'>
        <meta name='keywords' content='zranitelnost krajiny, georeport, ekologie'>
        <meta name='author' content='Adéla Petřeková'>
        <style>
            @page {{
                size: A4;
                margin: 10mm 20mm;
                @bottom-center {{
                    content: "Strana " counter(page);
                    font-family: Arial, sans-serif;
                    text-align: center; /* Uložení číslování strany na střed */
                }}
            }}
            body {{
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
            }}
            
            h1 {{
                background-color: #4CAF50;
                color: #fff;
                padding: 10px;
            }}
            p {{
                padding: 10px;
            }}
            .footer {{
                position: fixed;
                bottom: 0;
                width: 100%;
                text-align: center;
            }}
            .footer hr {{
                width: 100%;
                border: none;
                border-top: 1px solid #000;
                margin: 0;
            }}
            .footer-content {{
                display: flex;
                justify-content: space-between;
                align-items: center;
                padding: 0 0;
            }}
            .footer img {{
                height: 50px;
            }}
            .page_number {{
                font-family: Arial, sans-serif;
            }}
            .indicator-box {{
                border: 2px solid #4CAF50;
                padding: 10px;
                margin: 10px 0;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
            }}
            th, td {{
                border: 1px solid #000;
                padding: 8px;
                text-align: left;
            }}
            th {{
                background-color: #f2f2f2;
            }}
            
        </style>
    </head>
    <body>
    <header>
        <h1>Název reportu: Zranitelnost krajiny</h1>
        <p>Lokalita: XXX (GPS souřadnice) <br>
        Souřadnice: MBR extentu <br>
        Datum a čas generování: {timestamp} <br>
        Unikatní identifikátor: {id}
        </p>
        </header>
        <main>
        <p> Tento report podává zprávu o degradaci krajiny na základě metody ESAI ....</p>

        <h2> Krok I: Náchylnost krajiny k degradaci </h2>
        <p> Ve zvoleném území dosahuje výsledná hodnota náchylnosti k degradaci hodnoty "XX" a je klasifikována jako "textový název kategorie (číslo kategorie)"</p>

        <table>
            <tr>
                <th>Téma</th>
                <th>Hodnota</th>
                <th>Kategorie</th>
            </tr>
        """

    # Vložení dat do tabulky náchylnosti k degradaci
    for nadrazene_tema, data_degradace in serazene_nadrazene_degradace:
        html += f"""
        <tr>
            <td><strong>{nadrazene_tema}</strong></td>
            <td>{data_degradace['hodnota']}</td>
            <td>{data_degradace['kategorie']}</td>
        </tr>
        """
        for podrazene_tema, hodnota, kategorie in sorted(data_degradace['podrazene'], key=lambda x: -x[1]):
            html += f"""
            <tr>
                <td>{podrazene_tema}</td>
                <td>{hodnota}</td>
                <td>{kategorie}</td>
            </tr>
            """

    html += """
    </table>

    <div class="indicator-box">
            Nejhorší 3 indikátory v zájmovém území jsou: """

    # Nalezení a vložení 3 nejhorších indikátorů
    nejhorsi_indikatory = []
    for nadrazene_tema, data_degradace in serazene_nadrazene_degradace:
        serazene_podkategorie = sorted(data_degradace['podrazene'], key=lambda x: -x[1])
        nejhorsi_indikatory.extend([f"<em>{podrazene_tema}</em>" for podrazene_tema, _, _ in serazene_podkategorie[:3]])

    html += ", ".join(nejhorsi_indikatory[:3]) + "."

    html += """
        </div>
            <!-- vložení sloupcového grafu -->
            <h3> Věrohodnost dat </h3>
                <table>
                    <tr>
                        <th> Zkratka </th>
                        <th> Název </th>
                        <th> Datový zdroj </th>
                        <th> Garant </th>
                        <th> Měřítko </th>
                        <th> Rok aktualizace </th>
                    </tr>
    """

    # Přidání měřítek do tabulky
    for zkratka_txt, datovy_zdroj, indikator_cz, meritko, last_update, source_dat in data:
        if "1:" in meritko:
            barva, poznamka = urci_barvu_klasicke_meritko(meritko)
        elif "m/px" in meritko:
            barva, poznamka = urci_barvu_meritko_na_pixel(meritko)
        
        # Převedení last_update na datetime objekt
        barva_rok = urci_barvu_rok(last_update)

        html += f"""
                    <tr>
                        <td> {zkratka_txt} </td>
                        <td> {indikator_cz} </td>
                        <td> {datovy_zdroj} </td>
                        <td> {source_dat}  </td>
                        <td style="color: {barva};"> {meritko} {poznamka} </td>
                        <td style="color: {barva_rok};"> {last_update} </td>
                    </tr>
        """

    html += """
                </table>
                <!-- mapa území s celkovou kategorií -->

        <h2> Krok II: Posouzení funkčnosti krajiny </h2>
            <p> slouží k nalezení krajinných segmentů jejichž degradace bude mít velký/malý význam dopad na funkčnost krajiny </p>
            <table>
                <tr>
                    <th> </th>
                    <th> Hodnota </th>
                    <th> Kategorie </th>
                </tr>
                <tr>
                    <td> Zásoba uhlíku ve vegetaci </td>
                    <td> Hodnota 1 </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Evapotranspirace </td>
                    <td> Hodnota 1 </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Ekologická hodnota biotopu </td>
                    <td> Hodnota 1 </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Vodoretence </td>
                    <td> Hodnota 1 </td>
                    <td> Kategorie 1 </td>
                </tr>
                <tr>
                    <td> Integrovaná hodnota </td>
                    <td> --- </td>
                    <td> Kategorie 1 </td>
                </tr>
            </table>
            <!-- vložení map - 5x  -->
                <h3> Věrohodnost dat </h3>
                <table>
                    <tr>
                        <th> Název </th>
                        <th> Datový zdroj geometrie/KB</th>
                        <th> Garant </th>
                        <th> Měřítko </th>
                        <th> Rok aktualizace </th>
                    </tr>
   
                # Přidání měřítek do tabulky
   # for meritko in meritka:
    #    if "1:" in meritko:
     #       barva, poznamka = urci_barvu_klasicke_meritko(meritko)
      #  elif "m/px" in meritko:
       #     barva, poznamka = urci_barvu_meritko_na_pixel(meritko)
        #barva_rok = urci_barvu_rok(rok)

        
                    <tr>
                        <td> Indikátor 1 </td>
                        <td> Hodnota 1 </td>
                        <td> Věrohodnost 1 </td>
                        <td>  meritko</td>
                        <td>ROK </td>
                    </tr>
        """

    html += """
                </table>

        <h2> Krok III: Odolnost krajiny </h2>
            <!-- vložení mapy odolnosti -->

        <h2> Krok IV: Syntéze/lokalita prioritních opatření </h2>
            <!-- vložení mapy kde prioritně umístit opatření -->    

        </main>
        <footer>
            <div class="footer">
                <hr>
                <div class="footer-content">
                    <img src="katedra_logo.png" alt="Logo KGI">
                    <img src="logo_czechglobe.png" alt="Logo CzechGlobe">
                </div>
             </div>
        </footer>    
    </body>
    </html>
    """
 # Zde můžete uložit HTML do souboru nebo jej jinak zpracovat
    with open("report.html", "w", encoding="utf-8") as file:
        file.write(html)

    print("HTML report byl úspěšně vygenerován.")

except Exception as error:
    print(f"Chyba při připojování k databázi: {error}")

finally:
    if connection:
        cursor.close()
        connection.close()

Připojení k databázi bylo úspěšné

HTML report byl úspěšně vygenerován.

In [20]:
#vytvoření HTML souboru ze šablony
with open("index.html", "w") as f:
    f.write(html.format(timestamp=timestamp, id=id))
#otevření html souboru
#print("Generated index.html")
#import webbrowser
#webbrowser.open("index.html")

In [21]:
# output html as a PDF
# first, import the necessary modules

from weasyprint import HTML
# then, convert the HTML to a PDF
HTML("index.html").write_pdf("index.pdf")

print("Generated index.pdf")
#otevření pdf souboru
import webbrowser
webbrowser.open("index.pdf")

Generated index.pdf

True